In [30]:
import csv
from typing import Optional

import arrow
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import Sequential
from keras.src.layers import LSTM, Dropout, Dense
from pandas import DatetimeIndex
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# 可视化结果（如果需要）
import matplotlib.pyplot as plt
import os
import pathlib
import xml.etree.ElementTree as ET
import xarray as xr
import codecs
import datetime

# 先从海浪数据中提取出经纬度，时间，风，海浪高度
# 解析单个文件，并存于字典内
from sklearn.preprocessing import StandardScaler
# from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler


In [31]:

def batch_get_realdata(file_full_path: str, split_hours=72, issue_hours_steps: int = 12) -> pd.DataFrame:
    """
        TODO:[-] 25-04-23 生成实况训练数据集
        从指定文件批量获取时间数据并以dataframe的形式返回
    :param file_full_path:
    :return:
    """

    """
        eg: csv文件样例:
                        time	longitude	latitude	WS	YBG
                        202401010000
                        YYYYMMDDHHmm
    """
    list_series = []
    merge_dict = {}
    if pathlib.Path(file_full_path).exists():
        # ds: xr.Dataset = xr.open_dataset(file_full_path)
        df: pd.DataFrame = pd.read_csv(file_full_path)
        """读取指定路径的浮标处理后的一年的数据"""
        # 通过起止时间找到对应的index，然后每次的发布时间间隔步长为12h

        # step1: 生成2024年一年的时间步长为1hour的时间索引集合
        # TODO:[-] 25-05-12 注意此部分修改起止时间 local time -> utc time
        start_time = '2024-01-01 00:00:00'
        end_time = '2024-12-31 23:00:00'
        start_time_utc = '2023-12-31 16:00:00'
        end_time_utc = '2024-12-31 15:00:00'
        time_series = pd.date_range(start=start_time, end=end_time, freq='H')

        # 将time列的内容从int64 => str
        df['time'] = df['time'].astype(str)
        df['time'] = pd.to_datetime(df['time'], format='%Y%m%d%H%M')
        # step2: 将 time列设置为index，并将index替换为标准时间集合
        df.set_index('time', inplace=True)
        # TODO:[-] 25-05-12 注意直接重设index会造成所有数据为 nan 的情况。 解决办法：先创建新的utc_times 再设置为索引列
        # ValueError: Length of values (8784) does not match length of index (8887)

        df_reindexed = df.reindex(time_series)
        utc_times = pd.date_range(start=start_time_utc, end=end_time_utc, freq='H', tz='utc')
        df_reindexed['utc_times'] = utc_times
        df_reindexed = df_reindexed.set_index('utc_times')
        # 此处不需要再删除 time了，因为 index 已经 utc_times -> index了
        # df_reindexed = df_reindexed.drop('time', axis=1)
        df_reindexed.index.name = 'time'

        # step3: 生成12小时为间隔的时间数组
        freq_str: str = f'{issue_hours_steps}H'
        start_time_split_utc: str = '2024-01-01 00:00:00'
        end_time_split_utc: str = '2024-12-31 23:00:00'
        issue_dt_series = pd.date_range(start=start_time_split_utc, end=end_time_split_utc, freq=freq_str, tz='utc')

        for temp_time in issue_dt_series:
            temp_index: int = df_reindexed.index.get_loc(temp_time)
            val_series = df_reindexed[temp_index:temp_index + split_hours]
            list_series.append(val_series)
        # TODO:[-] 25-04-24 此处做重新修改，拼接成一个dataframe

        for temp_time in issue_dt_series:
            dt_str: str = temp_time.strftime('%Y%m%d%H%M%S')
            temp_index: int = df_reindexed.index.get_loc(temp_time)
            val_series = df_reindexed[temp_index:temp_index + split_hours]
            # 此处改为只取 'WS' 列
            # TODO:[-] 25-04-24 住一次此处需要将每一个 series的index索引重置为 [0,71]
            merge_dict[dt_str] = val_series['WS'].reset_index(drop=True)
            # list_series.append(val_series)
    df = pd.DataFrame.from_dict(merge_dict)
    # TODO:[-] 25-05-11 注意此处的 columns 为 字符串，需要将 str -> datetime
    df.columns = pd.to_datetime(df.columns)
    return df



In [32]:

def get_test_array(test_read_path: str, training_read_path: str, issue_times_index: DatetimeIndex):
    """
        分别读取测试数据集以及实况数据集并进行训练
    :param test_read_path:
    :param training_read_path:
    :return:
    """
    if pathlib.Path(test_read_path).exists() and pathlib.Path(training_read_path).exists():
        df_test: pd.DataFrame = pd.read_csv(test_read_path)
        u_data_dict = {}
        v_data_dict = {}
        # 读取的预报风场——测试训练集 在 df 中是通过 xxx_u与 xxx_v 的形式进行存储
        # TODO:[-] 25-04-28 u 与 v 每个共613组预报数据
        for col_name in df_test.columns:
            try:
                col_vector = df_test[col_name]
                # yyyymmddhhss
                dt_temp_str: str = col_name.split('_')[0]
                # u or v
                var_temp_str: str = col_name.split('_')[1]
                if var_temp_str == 'u':
                    # u_data_dict[dt_temp_str] = col_vector.tolist()
                    u_data_dict[dt_temp_str] = col_vector
                elif var_temp_str == 'v':
                    # v_data_dict[dt_temp_str] = col_vector.tolist()
                    v_data_dict[dt_temp_str] = col_vector
                # print(f'当前列:{col_name}处理成功~')
            except Exception as e:
                print(f'当前列:{col_name}处理错误!')
        # # step2: 将字典统一转换为二维数组
        # result_u_array = [val for key, val in u_data_dict.items()]
        # result_v_array = [val for key, val in v_data_dict.items()]
        # return [result_u_array, result_v_array]
        df_u = pd.DataFrame.from_dict(u_data_dict)
        df_v = pd.DataFrame.from_dict(v_data_dict)
        # 将时间字符串=>datetime
        df_u.columns = pd.to_datetime(df_u.columns)
        df_v.columns = pd.to_datetime(df_v.columns)
        # TODO:[*] 25-04-29
        # 需要根据起止时间及时间步长，生成对应的时间索引，并将该时间索引作为标准索引
        # 注意： reindex 后会返回一个新的 DataFrame，并不会修改原始df
        df_u = df_u.reindex(columns=issue_times_index)
        df_v = df_v.reindex(columns=issue_times_index)
        return df_u, df_v
        # pass
    return None



In [47]:
read_path: str = r'Z:\WIND\GRAPES\2024'
out_put_path: str = r'./data'
out_put_file_path: str = str(pathlib.Path(out_put_path) / 'GRAPES_2024_24')
lat: float = 39.5003
lng: float = 120.59533
# TODO:[*] 25-04-29 根据起止时间，时间步长为12h，生成发布时间index
start_time = '2024-01-01 00:00:00'
end_time = '2024-12-31 23:00:00'
issue_times_index = pd.date_range(start=start_time, end=end_time, freq='12h')
# step1: 提取数据集
# mac nas 地址
test_read_path: str = r'/Volumes/DATA/01TRAINNING_DATA/WIND/merge.csv'
train_read_path: str = r'/Volumes/DATA/FUB/MF01001/2024_local.csv'
read_file_full_path: str = r'/Volumes/DATA/FUB/MF01001/2024_local.csv'
# win nas 地址
# read_file_full_path: str = r'G:\05DATA\01TRAINING_DATA\FUB\MF01001\2024_local.csv'
# test_read_path: str = r'G:\05DATA\01TRAINING_DATA\WIND\merge.csv'
# train_read_path: str = r'G:\05DATA\01TRAINING_DATA\FUB\MF01001\2024_local.csv'
df_u, df_v = get_test_array(test_read_path,
                            train_read_path, issue_times_index)
#  生成一年的 365*2 =730 个 ws,ybg -> 只取 ws
# 注意实况 shape : (72,732)
df_ws = batch_get_realdata(read_file_full_path)
"""shape:(72,732)"""
df_ws_subset = df_ws.iloc[:25, :]

当前列:Unnamed: 0.10处理错误!
当前列:Unnamed: 0处理错误!
当前列:Unnamed: 0.1处理错误!
当前列:Unnamed: 0.2处理错误!
当前列:Unnamed: 0.3处理错误!
当前列:Unnamed: 0.4处理错误!
当前列:Unnamed: 0.5处理错误!
当前列:Unnamed: 0.6处理错误!
当前列:Unnamed: 0.7处理错误!
当前列:Unnamed: 0.8处理错误!
当前列:Unnamed: 0.9处理错误!


/var/folders/6f/ftn9fkvs23n16zlzyp9m_yn00000gp/T/ipykernel_80665/2096832180.py:29: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_series = pd.date_range(start=start_time, end=end_time, freq='H')
/var/folders/6f/ftn9fkvs23n16zlzyp9m_yn00000gp/T/ipykernel_80665/2096832180.py:40: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  utc_times = pd.date_range(start=start_time_utc, end=end_time_utc, freq='H', tz='utc')
/var/folders/6f/ftn9fkvs23n16zlzyp9m_yn00000gp/T/ipykernel_80665/2096832180.py:51: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  issue_dt_series = pd.date_range(start=start_time_split_utc, end=end_time_split_utc, freq=freq_str, tz='utc')


In [48]:
min = 0
max = 30
df_ws_subset = df_ws_subset.where((df_ws_subset > min) & (df_ws_subset < max))

In [49]:
df_ws_forecast = np.sqrt(df_u ** 2 + df_v ** 2)

In [50]:
min = 0
max = 30
df_ws_forecast = df_ws_forecast.where((df_ws_forecast > min) & (df_ws_forecast < max))

In [51]:
df_ws_forecast.shape

(25, 732)

In [52]:
df_ws_subset.shape

(25, 732)

In [53]:
# 获取 实况 与 预报 含有 nan 的列数
valid_columns = df_ws_forecast.dropna(axis=1).columns

In [54]:
len(valid_columns)

612

以上可知有120列数据中包含空置，需要提出

In [40]:
df_ws_forecast.iloc[:,0].to_numpy()

array([ 2.92387806,  6.53082466,  8.88998947,  9.03952286,  8.6541489 ,
        9.03994932,  8.16863939,  6.40596658,  3.74354731,  4.21659571,
        4.50508296,  2.4987569 ,  1.19714345,  3.0430362 ,  4.6989031 ,
        5.67712525,  5.20769807,  3.13559842,  5.30012564,  7.34407257,
        7.49159227,  8.00045529,  9.33985399, 10.53265526, 11.21003988])

In [41]:
df_ws_subset.iloc[:,0].to_numpy()

array([ 4. ,  5.3,  6.4,  nan,  8.1,  9.3, 10.6,  9.8, 11.3, 11.9, 12.2,
       11.8, 12.8, 11.1,  9.9,  9.3, 10.4, 10.4, 10.8, 10.1,  9.4,  8.6,
        7.5,  7.4,  4.5])

In [42]:
y_pred=df_ws_forecast.iloc[:,144].to_numpy()
y_true=df_ws_subset.iloc[:,144].to_numpy()

In [43]:
y_pred

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [44]:
y_true

array([8.7, 7. , 6.7, nan, 4.7, 5.2, 5.1, 4. , 4.7, 4.3, 4.1, 2.5, 3.2,
       3.2, 2.3, 2.9, 4.5, 4. , 4.6, 3.5, 0.6, 2. , 3.8, 4.7, 4.4])

In [46]:
# 求 732 组 数据的 平均 RMSE
rmse_list = []
err_count=0  
for i in range(732):
    try:
        y_pred = df_ws_forecast.iloc[:, i].to_numpy()
        y_true = df_ws_subset.iloc[:, i].to_numpy()
        imputer = SimpleImputer(strategy="mean")
        y_true_imputed = imputer.fit_transform(y_true.reshape(-1, 1)).flatten()
        y_pred_imputed = imputer.fit_transform(y_pred.reshape(-1, 1)).flatten()
        rmse = np.sqrt(mean_squared_error(y_true_imputed, y_pred_imputed))
        rmse_list.append(rmse)
        # print(f"RMSE for column {i}:{rmse}")
    except Exception as e:
        err_count+=1
        
# 计算 RMSE 的平均值
print(f"Error processing column count:{err_count}")
average_rmse = np.mean(rmse_list)
print("Average RMSE:", average_rmse)

/Users/evaseemefly/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: [0]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/evaseemefly/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: [0]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/evaseemefly/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: [0]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/evaseemefly/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: [0]. At least one non-missing value is needed for imputation

Error processing column count:120
Average RMSE: 3.88441365819189


/Users/evaseemefly/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: [0]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/evaseemefly/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: [0]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/evaseemefly/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: [0]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/evaseemefly/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: [0]. At least one non-missing value is needed for imputation

In [55]:
# 求 732 组 数据的 平均 RMSE
rmse_list = []
err_count=0  
for i in range(732):
    try:
        y_pred = df_ws_forecast.iloc[:, i].to_numpy()
        y_true = df_ws_subset.iloc[:, i].to_numpy()
        imputer = SimpleImputer(strategy="mean")
        y_true_imputed = imputer.fit_transform(y_true.reshape(-1, 1)).flatten()
        y_pred_imputed = imputer.fit_transform(y_pred.reshape(-1, 1)).flatten()
        rmse = np.sqrt(mean_squared_error(y_true_imputed, y_pred_imputed))
        rmse_list.append(rmse)
        # print(f"RMSE for column {i}:{rmse}")
    except Exception as e:
        err_count+=1
        
# 计算 RMSE 的平均值
rmse_series = pd.Series(rmse_list)

/Users/evaseemefly/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: [0]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/evaseemefly/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: [0]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/evaseemefly/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: [0]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/evaseemefly/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: [0]. At least one non-missing value is needed for imputation

In [56]:
rmse_series

0      5.322095
1      6.195090
2      4.282610
3      3.691367
4      4.660014
         ...   
607    4.667387
608    3.918225
609    5.797693
610    5.187597
611    4.317615
Length: 612, dtype: float64

In [57]:
np.mean(rmse_list)

3.88441365819189